In [32]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold

In [33]:
train = pd.read_csv('mertrain.csv')
test= pd.read_csv('mertest.csv')

In [49]:
from numpy.random import choice
n_elements = train.shape[0]
feats = train.columns.values[9:] 
n_feats = len(feats) 
weights = np.array(np.zeros(n_feats))
weights += 1/n_feats

best_feats = [] 
best_score = 0 
epochs = 3000 
n_features = 10 
# try out different values
wt_g = .2 
#weight growth rate
w_threshold = .2
#weight to add feature
kf = KFold( n_splits=3, shuffle=True)
kf.get_n_splits(train.drop(['y'], axis=1).values)
y = train['y'].values 
scores = [] 
top_feats = np.array([])

In [52]:
for i, ind in enumerate(list(range(epochs))):
    sample_feat_ids = choice(a=n_feats, size=n_features, 
                         replace=False, p=weights)
    sample_feats = np.append(top_feats, feats[sample_feat_ids])
    tst_P = np.array(np.zeros(n_elements))

    for trn, tst in kf.split(train.drop(['y'],axis=1).values) : 
        X = train.loc[:,sample_feats]
        trn_X, tst_X = X.iloc[trn,:], X.iloc[tst,:]
        trn_Y, tst_Y = clipped_y[trn], clipped_y[tst]
        #mod = SVR(C=50, epsilon=3, gamma=.2)
        #mod = GBR(alpha=.01, n_estimators=50, max_depth=5, min_samples_leaf=15, subsample=.5, random_state=1776)
        mod = RFR(n_estimators=100, max_depth=12, max_features=20, min_samples_leaf=4, n_jobs=2, random_state=1776)
        mod.fit(trn_X, trn_Y)
        tst_P[tst] = mod.predict(tst_X)

    # I don't want to overfit to outliers so I am clipping all y's at 155
        tst_rsq = r2(y_pred=tst_P, y_true=clipped_y)

    if ind > 29:
        scores.append(tst_rsq)
        ma_rsq = np.mean(scores[-30:])
        if ind % 25 == 0:
            print(ind, ma_rsq, tst_rsq)
            if tst_rsq > ma_rsq:
                weights[sample_feat_ids] *= (1+wt_g)
                sum_w = np.sum(weights)
                weights /= sum_w
            if tst_rsq > best_score:
                best_score = tst_rsq
                best_feats = sample_feats
            else:
                weights[sample_feat_ids] *= (1-wt_g)
                sum_w = np.sum(weights)
                weights /= sum_w
                mx_w = np.max(weights)

        # add feature to the top feats if weight is > threshold
        if mx_w > w_threshold:
            feat_imps = pd.Series(index = feats, data = weights)
            new_feats = feat_imps[feat_imps > w_threshold].index.values
            top_weights = feat_imps[feat_imps > w_threshold].values
            top_feats = np.append(top_feats, new_feats)
            feats = list(feats)
            weights = list(weights)
            for f,w in zip(new_feats,top_weights):
                print(f, w)
                feats.remove(f)
                weights.remove(w)
                n_feats = len(weights)
                feats = np.array(feats)
                weights = np.array(weights)
                sum_w = np.sum(weights)
                weights /= sum_w

        if ind % 25 == 0: 
            print(mx_w)
    else:
        scores.append(tst_rsq)
        print(ind, tst_rsq)

NameError: name 'clipped_y' is not defined